In [ ]:
!nvidia-smi

In [ ]:
!pip install -U \
  transformers \
  peft \
  accelerate \
  bitsandbytes \
  trl \
  datasets \
  huggingface_hub

In [ ]:
import os

HF_TOKEN = "token yeta rakha"

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
os.environ["TRANSFORMERS_TOKEN"] = HF_TOKEN

print("HF_TOKEN set:", HF_TOKEN[:8] + "...")

In [ ]:
from transformers import AutoTokenizer, AutoConfig

model_id = "unsloth/gemma-3-12b-it-bnb-4bit"

# Tokenizer test
tok = AutoTokenizer.from_pretrained(model_id)
print("Tokenizer loaded")

# Config test
cfg = AutoConfig.from_pretrained(model_id)
print("Config loaded")

In [ ]:
%%writefile train_lora.py
import argparse
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

# -----------------------------
# Args
# -----------------------------
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name_or_path", type=str, required=True)
    parser.add_argument("--dataset_path", type=str, required=True)
    parser.add_argument("--output_dir", type=str, required=True)
    parser.add_argument("--max_seq_length", type=int, default=1024)
    return parser.parse_args()

# -----------------------------
# Main
# -----------------------------
def main():
    args = parse_args()

    # -----------------------------
    # Tokenizer
    # -----------------------------
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_name_or_path,
        trust_remote_code=True,
    )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # -----------------------------
    # Model (already 4-bit)
    # -----------------------------
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        device_map="auto",
        trust_remote_code=True,
    )

    model.config.use_cache = False
    model = prepare_model_for_kbit_training(model)
    model.gradient_checkpointing_enable()

    # -----------------------------
    # LoRA
    # -----------------------------
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # -----------------------------
    # Dataset
    # -----------------------------
    raw_ds = load_dataset(
        "json",
        data_files=args.dataset_path,
        split="train",
    )

    def format_and_tokenize(example):
        instruction = example["instruction"]
        user_input = example["input"]
        output = example["output"]
    
        prompt = (
            f"{instruction}\n\n"
            f"{user_input}\n\n"
            "Answer:\n"
        )
    
        prompt_ids = tokenizer(
            prompt,
            truncation=True,
            max_length=args.max_seq_length,
            add_special_tokens=False,
        )["input_ids"]
    
        answer_ids = tokenizer(
            output,
            truncation=True,
            max_length=args.max_seq_length - len(prompt_ids),
            add_special_tokens=False,
        )["input_ids"]
    
        input_ids = prompt_ids + answer_ids
        labels = [-100] * len(prompt_ids) + answer_ids
    
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": [1] * len(input_ids),
            # REQUIRED FOR GEMMA-3
            "token_type_ids": [0] * len(input_ids),
        }


    dataset = raw_ds.map(
        format_and_tokenize,
        remove_columns=raw_ds.column_names,
        desc="Tokenizing dataset",
    )

    # -----------------------------
    # Training
    # -----------------------------
    training_args = TrainingArguments(
        output_dir=args.output_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        save_strategy="epoch",
        save_total_limit=1,
        report_to="none",
        optim="paged_adamw_8bit",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    trainer.train()

    model.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)

    print("LoRA training finished correctly")

if __name__ == "__main__":
    main()

In [ ]:
!ls /kaggle/input/askm-processed-datasets

In [ ]:
!accelerate launch \
  --num_processes=1 \
  train_lora.py \
  --model_name_or_path unsloth/gemma-3-12b-it-bnb-4bit \
  --dataset_path /kaggle/input/askm-processed-datasets/exam_lora.jsonl \
  --output_dir /kaggle/working/lora_outputs/exam_lora \
  --max_seq_length 1024

In [ ]:
!ls /kaggle/working/lora_outputs/exam_lora

In [ ]:
!zip -r exam_lora.zip /kaggle/working/lora_outputs/exam_lora